In [1]:
from transformers import DistilBertForSequenceClassification,DistilBertTokenizer
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, TensorDataset,DataLoader
from sklearn.model_selection import train_test_split
import torch
torch.__version__

I0307 16:01:56.332556 4446334464 file_utils.py:41] PyTorch version 1.7.1 available.


'1.7.1'

In [6]:
# Load DistilBert Classification model from transformer library
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=1)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load saved best BERT model op CPU
device = torch.device('cpu')
model.load_state_dict(torch.load('DistilBERT_best_model.pt', map_location=device))
model.eval()

I0307 16:05:15.470422 4446334464 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /Users/Bart/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
I0307 16:05:15.473039 4446334464 configuration_utils.py:292] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": null,
  "dim": 768,
  "do_sample": false,
  "dropout": 0.1,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 1

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
# get list of all paths to the json-files of english episodes given subset number (bart: 0 , juno: 1, joris: 2)
def get_paths_for_en_episodes(subset_number):
    """
    Function returns list of all paths to the json-files of english 
    episodes given subset number (bart: 0 , juno: 1, joris: 2)
    
    """
    metadata_df = pd.read_csv("podcast_data_no_audio/metadata/metadata.tsv",sep='\t')
    path1 = 'podcast_data_no_audio/podcasts-transcripts/' + str(subset_number)

    folders = listdir(path1)

    if '.DS_Store' in folders:
        folders.remove('.DS_Store')

    podcast_episodes_paths = []

    for letter_or_number in tqdm(folders):    
        path2 = path1 + '/' + letter_or_number


        for show_uri in listdir(path2):
            path3 = path2 + '/' + show_uri

            # select english shows only
            show_metadata = metadata_df.loc[metadata_df['show_filename_prefix'] == show_uri]

            if len(show_metadata['language'].unique()) > 0:
                if 'en' in show_metadata['language'].unique()[0]:
                    for episode_uri in listdir(path3):
                        path4 = path3 + '/' + episode_uri

                        if '.json' in path4:
                            podcast_episodes_paths.append(path4)

                
        
    return len(podcast_episodes_paths), podcast_episodes_paths

In [7]:
# load podcast metadata
metadata_df = pd.read_csv("podcast_data_no_audio/metadata/metadata.tsv",sep='\t')
metadata_df.head(1)

# Load all english episodes in dataset (so 0, 1 and 2 combined)


,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix
0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj


In [4]:
tokenized_utterances = train_df.text.apply(lambda x: tokenizer.encode(x,add_special_tokens=True))
tokenized_utterances_test = test_df.text.apply(lambda x: tokenizer.encode(x,add_special_tokens=True))

In [5]:
# pad sentences for training set
max_len = max(map(len,tokenized_utterances))
padded_utterances = np.array([ i+[0]*(max_len-len(i))  for i in tokenized_utterances])
attention_masked_utterances = np.where(padded_utterances != 0,1,0)

In [6]:
# pad sentences for test set
padded_utterances_test = np.array([ i+[0]*(max_len-len(i))  for i in tokenized_utterances_test])
attention_masked_utterances_test = np.where(padded_utterances_test != 0,1,0)

In [7]:
# Create tensors

X_train = torch.tensor(padded_utterances)
X_train_attention = torch.tensor(attention_masked_utterances)
y_train = torch.tensor(np.array(train_df.sentiment_score.values)[:,np.newaxis], dtype=torch.float32)

X_test = torch.tensor(padded_utterances_test)
X_test_attention = torch.tensor(attention_masked_utterances_test)
y_test = torch.tensor(np.array(test_df.sentiment_score.values)[:,np.newaxis], dtype=torch.float32)


train_data = TensorDataset(X_train, X_train_attention, y_train)
train_loader = DataLoader(train_data,batch_size=16, shuffle=True)

test_dataset = TensorDataset(X_test, X_test_attention)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       